In [1]:
import numpy as np
import cv2

In [7]:
def on_click(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        params.append((x, y))

In [19]:
def get_correspondences(im1, im2):
    
    out = []
    
    for i, im in enumerate([im1, im2]):
        
        points = []
        cv2.namedWindow('image')
        cv2.setMouseCallback('image', on_click, points)

        while (True):
            cv2.imshow('image', im)
            if cv2.waitKey(20) == 27:
                break

        cv2.destroyAllWindows()
        
        out.append(list(points))
        
    return out[0], out[1]

In [20]:
im1 = cv2.imread('crop1.jpg')
im2 = cv2.imread('crop2.jpg')

In [21]:
t1, t2 = get_correspondences(im1, im2)

In [22]:
t

[[(366, 262)], [(124, 356)]]

In [12]:
x = [[(1, 2)], [(3, 4)]]

In [13]:
a, b = x

In [4]:
im = cv2.imread('crop1.jpg')
cv2.namedWindow('image')
cv2.setMouseCallback('image', on_click)

while (True):
    cv2.imshow('image', im)
    if cv2.waitKey(20) == 27:
        break

cv2.destroyAllWindows()

In [5]:
points

[(287, 201), (217, 214), (237, 198), (121, 128), (65, 150)]